In [1]:
!pip install git+https://github.com/pytorch/fairseq@336942734c85791a90baa373c212d27e7c722662#egg=fairseq

  Using cached fairseq-1.0.0a0+3369427-cp39-cp39-macosx_10_9_x86_64.whl


In [13]:
!pip install numpy==1.23.3

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.1/18.1 MB 12.7 MB/s eta 0:00:0000:0100:01
  Attempting uninstall: numpy
    Found existing installation: numpy 1.24.3
    Uninstalling numpy-1.24.3:
      Successfully uninstalled numpy-1.24.3


In [ ]:
!pip install gdown

In [2]:
import numpy as np

In [10]:
import torch
import torch.nn.functional as F
import sys

In [5]:
from fairseq.models.roberta import RobertaModel

In [4]:
from musicbert import *

disable_cp = False
mask_strategy = ['bar']
convert_encoding = OCTMIDI
crop_length = None


In [13]:
bar_max = 256 #@param {type:"integer"}
pos_resolution = 16  # per beat (quarter note)
velocity_quant = 4
tempo_quant = 12  # 2 ** (1 / 12)
min_tempo = 16
max_tempo = 256
duration_max = 8  # 2 ** 8 * beat
max_ts_denominator = 6  # x/1 x/2 x/4 ... x/64
max_notes_per_bar = 2  # 1/64 ... 128/64
beat_note_factor = 4  # In MIDI format a note is always 4 beats
deduplicate = True
filter_symbolic = False
filter_symbolic_ppl = 16
trunc_pos = 2 ** 16  # approx 30 minutes (1024 measures)
sample_len_max = 1000  # window length max
sample_overlap_rate = 4
ts_filter = False
pool_num = 24
max_inst = 127
max_pitch = 127
max_velocity = 127

In [17]:
def t2e(x):
    assert x in ts_dict, 'unsupported time signature: ' + str(x)
    return ts_dict[x]


def e2t(x):
    return ts_list[x]


def d2e(x):
    return dur_enc[x] if x < len(dur_enc) else dur_enc[-1]


def e2d(x):
    return dur_dec[x] if x < len(dur_dec) else dur_dec[-1]


def v2e(x):
    return x // velocity_quant


def e2v(x):
    return (x * velocity_quant) + (velocity_quant // 2)


def b2e(x):
    x = max(x, min_tempo)
    x = min(x, max_tempo)
    x = x / min_tempo
    e = round(math.log2(x) * tempo_quant)
    return e


def e2b(x):
    return 2 ** (x / tempo_quant) * min_tempo

In [18]:
def gen_dictionary(file_name):
    num = 0
    with open(file_name, 'w') as f:
        for j in range(bar_max):
            print('<0-{}>'.format(j), num, file=f)
        for j in range(beat_note_factor * max_notes_per_bar * pos_resolution):
            print('<1-{}>'.format(j), num, file=f)
        for j in range(max_inst + 1 + 1):
            # max_inst + 1 for percussion
            print('<2-{}>'.format(j), num, file=f)
        for j in range(2 * max_pitch + 1 + 1):
            # max_pitch + 1 ~ 2 * max_pitch + 1 for percussion
            print('<3-{}>'.format(j), num, file=f)
        for j in range(duration_max * pos_resolution):
            print('<4-{}>'.format(j), num, file=f)
        for j in range(v2e(max_velocity) + 1):
            print('<5-{}>'.format(j), num, file=f)
        for j in range(len(ts_list)):
            print('<6-{}>'.format(j), num, file=f)
        for j in range(b2e(max_tempo) + 1):
            print('<7-{}>'.format(j), num, file=f)


In [20]:
ts_dict = dict()
ts_list = list()
for i in range(0, max_ts_denominator + 1):  # 1 ~ 64
    for j in range(1, ((2 ** i) * max_notes_per_bar) + 1):
        ts_dict[(j, 2 ** i)] = len(ts_dict)
        ts_list.append((j, 2 ** i))
dur_enc = list()
dur_dec = list()
for i in range(duration_max):
    for j in range(pos_resolution):
        dur_dec.append(len(dur_enc))
        for k in range(2 ** i):
            dur_enc.append(len(dur_dec) - 1)

In [30]:
!mkdir -p input0
ROOT = '/Users/huongngo/Desktop/RESEARCH/text-to-midi/muzic/musicbert/input0' 
gen_dictionary('{}/dict.txt'.format(ROOT))

In [31]:
!mkdir -p label
ROOT = '/Users/huongngo/Desktop/RESEARCH/text-to-midi/muzic/musicbert/label' 
gen_dictionary('{}/dict.txt'.format(ROOT))

In [27]:
!gdown --id 1-BW45Vu80GU86ejTT4PFkgjvDhhYWatS

/Users/huongngo/opt/anaconda3/envs/temp/lib/python3.9/site-packages/gdown/cli.py:126: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From (uriginal): https://drive.google.com/uc?id=1-BW45Vu80GU86ejTT4PFkgjvDhhYWatS
From (redirected): https://drive.google.com/uc?id=1-BW45Vu80GU86ejTT4PFkgjvDhhYWatS&confirm=t&uuid=1caa71ee-3fe4-403f-85d6-67fe1018d831
To: /Users/huongngo/Desktop/RESEARCH/text-to-midi/muzic/musicbert/checkpoint_last_musicbert_small_w_genre_head.pt
100%|████████████████████████████████████████| 267M/267M [00:30<00:00, 8.77MB/s]


In [34]:
roberta_base = MusicBERTModel.from_pretrained('.', 
  checkpoint_file = '/Users/huongngo/Desktop/RESEARCH/text-to-midi/checkpoints/checkpoint_last_musicbert_small_w_genre_head.pt',
  user_dir='musicbert'    # activate the MusicBERT plugin with this keyword
)

FileNotFoundError: [Errno 2] No such file or directory: '/Users/huongngo/Desktop/RESEARCH/text-to-midi/muzic/musicbert/label/dict.txt'

In [33]:
print(roberta_base)

RobertaHubInterface(
  (model): MusicBERTModel(
    (encoder): MusicBERTEncoder(
      (sentence_encoder): OctupleEncoder(
        (dropout_module): FairseqDropout()
        (embed_tokens): Embedding(1237, 512, padding_idx=1)
        (embed_positions): LearnedPositionalEmbedding(8194, 512, padding_idx=1)
        (emb_layer_norm): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
        (layers): ModuleList(
          (0-3): 4 x TransformerSentenceEncoderLayer(
            (dropout_module): FairseqDropout()
            (activation_dropout_module): FairseqDropout()
            (self_attn): MultiheadAttention(
              (dropout_module): FairseqDropout()
              (k_proj): Linear(in_features=512, out_features=512, bias=True)
              (v_proj): Linear(in_features=512, out_features=512, bias=True)
              (q_proj): Linear(in_features=512, out_features=512, bias=True)
              (out_proj): Linear(in_features=512, out_features=512, bias=True)
            )
       

In [ ]:
roberta